In [2]:
import torch
import torchvision
import pandas
import numpy
import os
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomContrast, RandomBrightness, Flip, OneOf, Compose, RandomGamma, ElasticTransform, ChannelShuffle,RGBShift, Rotate
)
import cv2 as cv
from PIL import Image
from PIL.Image import Path
import colorsys

## Augmentation

In [ ]:
def augment(aug, image):
    return aug(image=image)['image']

def strong_aug(p=1):
    return Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=.1),
            Blur(blur_limit=3, p=.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomContrast(),
            RandomBrightness(),
        ], p=0.3),
        #HueSaturationValue(p=0.3),
    ], p=p)

aug = strong_aug(p=3)  

for i in range(1, len(os.listdir('c:\\users\\admin\\dataset\\1'))):
    for b in range(5):
        image = Image.open(Path('C:/Users/admin/dataset/1/image (' + str(i) + ').jpg'))
        new_image = Image.fromarray(augment(aug,np.array(image)))
        new_image.save('C:/Users/admin/dataset/1_aug/new_image (' + str(i) + '_' + str(b)+').jpg')  
        
for i in range(1, len(os.listdir('c:\\users\\admin\\dataset\\0'))):
    for b in range(5):
        image = Image.open(Path('C:/Users/admin/dataset/0/image (' + str(i) + ').jpg'))
        new_image = Image.fromarray(augment(aug,np.array(image)))
        new_image.save('C:/Users/admin/dataset/0_aug/new_image (' + str(i) + '_' + str(b)+').jpg')

In [ ]:
names_list = os.listdir('c:\\users\\admin\\dataset\\0\\')
counter = 1
for i in names_list:
#     im = Image.open(Path('C:/Users/admin/dataset/0/image (' + str(i) + ').jpg'))
#     ld = im.load()

#     width, height = im.size
#     for y in range(height):
#         for x in range(width):
#             r,g,b = ld[x,y]
#             h,s,v = colorsys.rgb_to_hsv(r/255., g/255., b/255.)

#             if s>0.5:                   
#                 ld[x,y] = (0,0,0)
#             else:
#                 ld[x,y] = (255,255,255)
#     im.save('C:/Users/admin/dataset/0_bin/new_image (' + str(i) + '_' + str(b)+').jpg')
    img = cv.imread('C:/Users/admin/dataset/0/' + i,0)
#     img = cv.medianBlur(img,5)
#     th3 = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
#             cv.THRESH_BINARY,11,2)
    #blur = cv.GaussianBlur(img,(5,5),0)
    ret2,th2 = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    cv.imwrite('C:/Users/admin/dataset/0_bin/new_image (' + str(counter) + ').jpg', th2)
    counter += 1

In [ ]:
names_list = os.listdir('c:\\users\\admin\\dataset\\1\\')
counter = 1
for i in names_list:
#     im = Image.open(Path('C:/Users/admin/dataset/0/image (' + str(i) + ').jpg'))
#     ld = im.load()

#     width, height = im.size
#     for y in range(height):
#         for x in range(width):
#             r,g,b = ld[x,y]
#             h,s,v = colorsys.rgb_to_hsv(r/255., g/255., b/255.)

#             if s>0.5:                   
#                 ld[x,y] = (0,0,0)
#             else:
#                 ld[x,y] = (255,255,255)
#     im.save('C:/Users/admin/dataset/0_bin/new_image (' + str(i) + '_' + str(b)+').jpg')
    img = cv.imread('C:/Users/admin/dataset/1/' + i,0)
#     img = cv.medianBlur(img,5)
#     th3 = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
#             cv.THRESH_BINARY,11,2)
    #blur = cv.GaussianBlur(img,(5,5),0)
    ret2,th2 = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    cv.imwrite('C:/Users/admin/dataset/1_bin/new_image (' + str(counter) + ').jpg', th2)
    counter += 1

## Load dataset

In [ ]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.RandomResizedCrop(390),
     torchvision.transforms.RandomRotation(degrees=15),
     torchvision.transforms.RandomHorizontalFlip(244),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
ngpu = 1
#device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

def load_dataset():
    data_path = 'dataset/'
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transform
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=25,
        num_workers=0,
        shuffle=True
    )
    return train_loader

dataloader = load_dataset()
classes = os.listdir('dataset/')

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


dataiter = iter(dataloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
resnet50 = models.resnet101(pretrained=True)

In [ ]:
for param in resnet50.parameters():
    param.requires_grad = False

In [ ]:
fc_inputs = resnet50.fc.in_features
 
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 390),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(390, 2),
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)

In [ ]:
#resnet50.load_state_dict(torch.load('c:\\users\\admin\\desktop\\resnet.pt'))
#resnet50.eval()
resnet50 = resnet50.to('cuda:0')

In [ ]:
loss_criterion = nn.NLLLoss()
optimizer = optim.Adam(resnet50.parameters())

In [ ]:
import time

for epoch in range(20):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch+1, 20))
     
    # Set to training mode
    resnet50.train()
     
    # Loss and Accuracy within the epoch
    train_loss = 0.0
    train_acc = 0.0
 
    for i, (inputs, labels) in enumerate(dataloader):
 
        inputs = inputs.to('cuda:0')
        labels = labels.to('cuda:0')
         
        # Clean existing gradients
        #optimizer.zero_grad()
         
        # Forward pass - compute outputs on input data using the model
        outputs = resnet50(inputs)
         
        # Compute loss
        loss = loss_criterion(outputs, labels)
         
        # Backpropagate the gradients
        loss.backward()
         
        # Update the parameters
        optimizer.step()
         
        # Compute the total loss for the batch and add it to train_loss
        train_loss += loss.item() * inputs.size(0)
         
        # Compute the accuracy
        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))
         
        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
         
        # Compute total accuracy in the whole batch and add to train_acc
        train_acc += acc.item() * inputs.size(0)
         
        print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

In [ ]:
torch.save(resnet50.state_dict(), 'c:\\users\\admin\\Desktop\\resnet_2.pt')

In [ ]:
from PIL import Image
from pathlib import Path

image = Image.open(Path('C:/Users/admin/desktop/example_1.jpg'))

In [ ]:
img = cv.imread('C:/Users/admin/desktop/example_1.jpg',0)
ret2,th2 = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
cv.imwrite('C:/Users/admin/desktop/example_new_1.jpg', th2)
image = Image.open(Path('C:/Users/admin/desktop/example_new_1.jpg'))

In [ ]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.RandomResizedCrop(390),
     #torchvision.transforms.RandomRotation(degrees=15),
     #torchvision.transforms.RandomHorizontalFlip(244),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

input = transform(image)
input = input.view(1, 3, 390, 390)

In [ ]:
resnet50 = models.resnet50(pretrained=True)
fc_inputs = resnet50.fc.in_features
 
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 244),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(244, 2),
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)
resnet50.load_state_dict(torch.load('c:\\users\\admin\\desktop\\resnet.pt'))
resnet50.eval()


In [ ]:
output = resnet50(input)

In [ ]:
prediction = int(torch.max(output.data, 1)[1].numpy())
print(prediction)

## pipeline

In [2]:
from PIL import Image
from pathlib import Path
import torchvision
import torch
import torchvision.models as models
import torch.nn as nn

image = Image.open(Path('C:/Users/admin/desktop/example_1.jpg'))
transform = torchvision.transforms.Compose(
    [torchvision.transforms.RandomResizedCrop(390),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

input = transform(image)
input = input.view(1, 3, 390, 390)

resnet50 = models.resnet50(pretrained=True)  #модель скачается автоматически если не скачана
fc_inputs = resnet50.fc.in_features
 
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 244),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(244, 2),
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)
resnet50.load_state_dict(torch.load('c:\\users\\admin\\desktop\\resnet.pt'))
resnet50.eval()

output = resnet50(input)
prediction = int(torch.max(output.data, 1)[1].numpy())  # результат